In [1]:
import pandas as pd
import numpy as np
from sql_functions import *
from normal_round import normal_round as normal_round

In [2]:
pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)

In [3]:
df_bund_abs = get_dataframe('SELECT * FROM public.bund_jugend_tat_absolut_2022_until_2018;')
df_bund_rel = get_dataframe('SELECT * FROM public.bund_jugend_tat_relativ_2022_until_2018;')

In [4]:
df_bund_abs.head(5)

,schluessel,straftat,sexus,tatverdaechtige_insgesamt,kinder_10_bis_unter_12,kinder_12_bis_unter_14,kinder_10_bis_unter_14,kinder_unter_14,jugendliche_14_bis_unter_16,jugendliche_16_bis_unter_18,jugendliche_14<18,heranwachsende_18_bis_unter_21,jugendl_u_heranwachsende_14_bis_unter_21,erwachsene_ab_21,year
0,------,Straftaten insgesamt,M,1541130,11882,25507,37389,48288,52244,76029,128273,147013,275286,1217556,2018
1,------,Straftaten insgesamt,W,510136,4377,12991,17368,22315,24451,24707,49158,38510,87668,400153,2018
2,------,Straftaten insgesamt,X,2051266,16259,38498,54757,70603,76695,100736,177431,185523,362954,1617709,2018
3,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,M,43036,310,1168,1478,1710,2861,2789,5650,4018,9668,31658,2018
4,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,W,2500,76,309,385,408,339,133,472,133,605,1487,2018


In [5]:
#df_bund_rel.tail()

### Calculate most occuring crime per state - rank and percentage of total

In [6]:
df_1 = df_bund_abs.copy()
df_1 = df_1[['schluessel', 'straftat', 'sexus', 'year', 'jugendliche_16_bis_unter_18']]
df_1 = df_1.rename(columns={'jugendliche_16_bis_unter_18': 'offenders'})

df_2 = df_1[df_1['schluessel'] == '------']
df_2 = df_2[['sexus', 'year', 'offenders']]
df_bund_straftaten_insg_absolut = df_2.copy() # needed later

df_3 = df_1[df_1['schluessel'] != '------']

df_3['rank'] = (
    df_3.groupby(['sexus', 'year'])['offenders']
        .rank(ascending=False, method='first')
        .astype(int)
)

df_4 = df_3.merge(df_2, on=['sexus', 'year'], how='left')

df_4['certain_crime_percent_of_total_crime'] = df_4['offenders_x'] / df_4['offenders_y'] * 100
df_4 = df_4.drop(columns='offenders_y')
df_4 = df_4.reset_index()
df_4 = df_4.drop(columns='index')

values = df_4['certain_crime_percent_of_total_crime'].tolist()
new_values = []
for v in values:
    new_v = normal_round(num=v, ndigits=2)
    new_values.append(new_v)
    
df_4['certain_crime_percent_of_total_crime'] = new_values

df_4 = df_4[['schluessel', 'sexus', 'year', 'straftat', 'rank', 'certain_crime_percent_of_total_crime']]

crime_rank_and_percent_of_total = df_4.copy()
crime_rank_and_percent_of_total.head()

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18460/1995338922.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['rank'] = (


,schluessel,sexus,year,straftat,rank,certain_crime_percent_of_total_crime
0,100000,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,6,3.67
1,100000,W,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,8,0.54
2,100000,X,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,6,2.90
3,210000,M,2018,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",5,3.86
4,210000,W,2018,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",5,0.98


### Create overview - number of offenders per state - absolut and relativ

In [7]:
df_10 = df_bund_rel.copy()
df_10 = df_10.rename(columns={'jugendliche_16_bis_unter_18_rel': 'offenders_rel'})
df_10 = df_10[(df_10['schluessel'] == '------')]

jugendkrim_insg_rel = df_10['offenders_rel'].tolist()
jugendkrim_insg_rel_rounded = []
for v in jugendkrim_insg_rel:
    v_new = int(normal_round(v, 0))
    jugendkrim_insg_rel_rounded.append(v_new)
df_10['offenders_rel'] = jugendkrim_insg_rel_rounded

df_10 = df_10[['sexus', 'year', 'offenders_rel']]
df_10 = df_10.reset_index()
df_10 = df_10.drop(columns='index')

df_11 = df_bund_straftaten_insg_absolut.merge(df_10, on=['sexus', 'year'], how='left')
df_bund_straftaten_insg_abs_u_rel = df_11.copy()
df_bund_straftaten_insg_abs_u_rel.head()

,sexus,year,offenders,offenders_rel
0,M,2018,76029,9105
1,W,2018,24707,3204
2,X,2018,100736,6272
3,M,2019,73445,9126
4,W,2019,23775,3172


### Merge crimes that <br>rank on 1, 2, and 3 <br>and <br>overall number of crimes - absolut and relativ <br>to every state for every year (->one DataFrame)

In [8]:
df_20 = crime_rank_and_percent_of_total.copy()
df_rank_1 = df_20[df_20['rank'] == 1]
df_rank_2 = df_20[df_20['rank'] == 2]
df_rank_3 = df_20[df_20['rank'] == 3]
df_rank_4 = df_20[df_20['rank'] == 4]
df_rank_5 = df_20[df_20['rank'] == 5]
df_rank_1.head()

,schluessel,sexus,year,straftat,rank,certain_crime_percent_of_total_crime
19,*26*00,W,2018,Ladendiebstahl insgesamt,1,29.11
24,730000,M,2018,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),1,27.10
26,730000,X,2018,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),1,24.01
49,*26*00,W,2019,Ladendiebstahl insgesamt,1,29.08
54,730000,M,2019,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),1,27.04


In [9]:
df_30 = df_bund_straftaten_insg_abs_u_rel.copy()

df_31 = df_30.merge(df_rank_1, on=['sexus', 'year'], how='left')
df_31 = df_31.rename(columns={'straftat': 'crimes_on_rank_1',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_1_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_1'})
df_31 = df_31.drop(columns='rank')

df_32 = df_31.merge(df_rank_2, on=['sexus', 'year'], how='left')
df_32 = df_32.rename(columns={'straftat': 'crimes_on_rank_2',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_2_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_2'})
df_32 = df_32.drop(columns='rank')

df_33 = df_32.merge(df_rank_3, on=['sexus', 'year'], how='left')
df_33 = df_33.rename(columns={'straftat': 'crimes_on_rank_3',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_3_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_3'})
df_33 = df_33.drop(columns='rank')

df_34 = df_33.merge(df_rank_4, on=['sexus', 'year'], how='left')
df_34 = df_34.rename(columns={'straftat': 'crimes_on_rank_4',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_4_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_4'})
df_34 = df_34.drop(columns='rank')

df_35 = df_34.merge(df_rank_5, on=['sexus', 'year'], how='left')
df_35 = df_35.rename(columns={'straftat': 'crimes_on_rank_5',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_5_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_5'})
df_35 = df_35.drop(columns='rank')

df_bund_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5 = df_35.copy() 
df_bund_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5.head()

,sexus,year,offenders,offenders_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
0,M,2018,76029,9105,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),27.10,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",24.01,*26*00,Ladendiebstahl insgesamt,13.68,674000,Sachbeschädigung §§ 303-305a StGB,12.84,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",3.86
1,W,2018,24707,3204,*26*00,Ladendiebstahl insgesamt,29.11,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",18.13,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),14.51,674000,Sachbeschädigung §§ 303-305a StGB,5.00,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",0.98
2,X,2018,100736,6272,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),24.01,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.57,*26*00,Ladendiebstahl insgesamt,17.46,674000,Sachbeschädigung §§ 303-305a StGB,10.92,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",3.15
3,M,2019,73445,9126,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),27.04,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",24.50,*26*00,Ladendiebstahl insgesamt,13.18,674000,Sachbeschädigung §§ 303-305a StGB,12.50,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,4.58
4,W,2019,23775,3172,*26*00,Ladendiebstahl insgesamt,29.08,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",17.97,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),14.65,674000,Sachbeschädigung §§ 303-305a StGB,4.76,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",1.02


### Calculate Growth Rate

In [10]:
df_51 = df_bund_abs.copy()
df_51 = df_51[["schluessel", "straftat", "year", 'sexus', "jugendliche_16_bis_unter_18"]]
df_51 = df_51[df_51["schluessel"] == "------"]
df_51 = df_51[["year", 'sexus', "jugendliche_16_bis_unter_18"]]

df_53 = pd.DataFrame(columns = ["year", 'sexus', "jugendliche_16_bis_unter_18", "growth_rate_abs"])

gender = ['X', 'W', 'M']

for g in gender:
    df_52 = df_51[df_51["sexus"] == g]
    df_52["growth_rate_abs"] = df_52['jugendliche_16_bis_unter_18'].pct_change() * 100
    df_53 = pd.concat([df_53, df_52])

df_53 = df_53.drop(columns='jugendliche_16_bis_unter_18')

df_bund_growth_rate_abs = df_53.copy()
df_bund_growth_rate_abs.head(5)

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18460/2383970582.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_52["growth_rate_abs"] = df_52['jugendliche_16_bis_unter_18'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18460/2383970582.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_52["growth_rate_abs"] = df_52['jugendliche_16_bis_unter_18'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18460/2383970582.py:12: Setti

,year,sexus,growth_rate_abs
2,2018,X,NaN
35,2019,X,-3.490311
68,2020,X,-6.659124
101,2021,X,-5.869129
134,2022,X,14.799813


In [11]:
df_61 = df_bund_rel.copy()
df_61 = df_61[["schluessel", "straftat", "year", 'sexus', "jugendliche_16_bis_unter_18_rel"]]
df_61 = df_61[df_61["schluessel"] == "------"]
df_61 = df_61[["year", 'sexus', "jugendliche_16_bis_unter_18_rel"]]

df_63 = pd.DataFrame(columns = ["year", 'sexus', "jugendliche_16_bis_unter_18_rel", "growth_rate_rel"])

gender = ['X', 'W', 'M']

for g in gender:
    df_62 = df_61[df_61["sexus"] == g]
    df_62["growth_rate_rel"] = df_62['jugendliche_16_bis_unter_18_rel'].pct_change() * 100
    df_63 = pd.concat([df_63, df_62])

df_63 = df_63.drop(columns='jugendliche_16_bis_unter_18_rel')

df_bund_growth_rate_rel = df_63.copy()
df_bund_growth_rate_rel.head(5)

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18460/1669702555.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_62["growth_rate_rel"] = df_62['jugendliche_16_bis_unter_18_rel'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18460/1669702555.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_62["growth_rate_rel"] = df_62['jugendliche_16_bis_unter_18_rel'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18460/1669702555.py:1

,year,sexus,growth_rate_rel
2,2018,X,NaN
35,2019,X,-0.268462
68,2020,X,-5.167812
101,2021,X,-5.380887
134,2022,X,15.225304


In [12]:
df_growth_rate_16_to_under_18 = df_bund_growth_rate_abs.merge(df_bund_growth_rate_rel, on=['year', 'sexus'], how='left')
df_growth_rate_16_to_under_18.head()

,year,sexus,growth_rate_abs,growth_rate_rel
0,2018,X,NaN,NaN
1,2019,X,-3.490311,-0.268462
2,2020,X,-6.659124,-5.167812
3,2021,X,-5.869129,-5.380887
4,2022,X,14.799813,15.225304


In [13]:
values_growth_rate_abs = df_growth_rate_16_to_under_18["growth_rate_abs"].tolist()
new_values_abs = []
for v in values_growth_rate_abs:
    if np.isnan(v):
        new_v = v
    else: 
        new_v = normal_round(v, 2)
    new_values_abs.append(new_v)
df_growth_rate_16_to_under_18["growth_rate_abs"] = new_values_abs

values_growth_rate_rel = df_growth_rate_16_to_under_18["growth_rate_rel"].tolist()
new_values_rel = []
for v in values_growth_rate_rel:
    if np.isnan(v):
        new_v = v
    else: 
        new_v = normal_round(v, 2)
    new_values_rel.append(new_v)
df_growth_rate_16_to_under_18["growth_rate_rel"] = new_values_rel

In [14]:
df_growth_rate_16_to_under_18.head()

,year,sexus,growth_rate_abs,growth_rate_rel
0,2018,X,NaN,NaN
1,2019,X,-3.48,-0.26
2,2020,X,-6.65,-5.16
3,2021,X,-5.86,-5.37
4,2022,X,14.80,15.23


In [15]:
df_overview_state_16_to_under_18 = df_bund_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5.merge(df_growth_rate_16_to_under_18, on=['sexus', 'year'], how='left')

df_overview_state_16_to_under_18['age_group'] = 'jugendliche_16_bis_unter_18'

# bring in correct order
df_overview_state_16_to_under_18 = df_overview_state_16_to_under_18[['year', 'sexus', 'age_group', 'offenders', 'offenders_rel',
       'growth_rate_abs', 'growth_rate_rel',
       'schluessel_crimes_on_rank_1', 'crimes_on_rank_1',
       'percentage_of_rank_1_on_crime_total', 'schluessel_crimes_on_rank_2',
       'crimes_on_rank_2', 'percentage_of_rank_2_on_crime_total',
       'schluessel_crimes_on_rank_3', 'crimes_on_rank_3',
       'percentage_of_rank_3_on_crime_total', 'schluessel_crimes_on_rank_4',
       'crimes_on_rank_4', 'percentage_of_rank_4_on_crime_total',
       'schluessel_crimes_on_rank_5', 'crimes_on_rank_5',
       'percentage_of_rank_5_on_crime_total']]

df_overview_state_16_to_under_18.tail(5)

,year,sexus,age_group,offenders,offenders_rel,growth_rate_abs,growth_rate_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
10,2021,W,jugendliche_16_bis_unter_18,20172,2734,-6.75,-6.43,*26*00,Ladendiebstahl insgesamt,24.65,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",16.90,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),15.15,674000,Sachbeschädigung §§ 303-305a StGB,5.52,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.69
11,2021,X,jugendliche_16_bis_unter_18,85420,5613,-5.86,-5.37,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),23.45,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.01,*26*00,Ladendiebstahl insgesamt,13.28,674000,Sachbeschädigung §§ 303-305a StGB,10.73,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,8.97
12,2022,M,jugendliche_16_bis_unter_18,73848,9447,13.18,13.52,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.82,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),20.24,*26*00,Ladendiebstahl insgesamt,13.55,674000,Sachbeschädigung §§ 303-305a StGB,10.57,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,9.87
13,2022,W,jugendliche_16_bis_unter_18,24214,3296,20.04,20.58,*26*00,Ladendiebstahl insgesamt,36.05,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",17.80,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),11.33,674000,Sachbeschädigung §§ 303-305a StGB,4.50,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.01
14,2022,X,jugendliche_16_bis_unter_18,98062,6467,14.80,15.23,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",21.58,*26*00,Ladendiebstahl insgesamt,19.11,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),18.04,674000,Sachbeschädigung §§ 303-305a StGB,9.07,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,8.18


In [16]:
df_overview_state_16_to_under_18 = df_overview_state_16_to_under_18.fillna('n.a.')

In [17]:
df_overview_state_16_to_under_18.head()

,year,sexus,age_group,offenders,offenders_rel,growth_rate_abs,growth_rate_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
0,2018,M,jugendliche_16_bis_unter_18,76029,9105,n.a.,n.a.,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),27.10,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",24.01,*26*00,Ladendiebstahl insgesamt,13.68,674000,Sachbeschädigung §§ 303-305a StGB,12.84,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",3.86
1,2018,W,jugendliche_16_bis_unter_18,24707,3204,n.a.,n.a.,*26*00,Ladendiebstahl insgesamt,29.11,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",18.13,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),14.51,674000,Sachbeschädigung §§ 303-305a StGB,5.00,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",0.98
2,2018,X,jugendliche_16_bis_unter_18,100736,6272,n.a.,n.a.,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),24.01,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.57,*26*00,Ladendiebstahl insgesamt,17.46,674000,Sachbeschädigung §§ 303-305a StGB,10.92,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",3.15
3,2019,M,jugendliche_16_bis_unter_18,73445,9126,-3.39,0.23,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),27.04,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",24.50,*26*00,Ladendiebstahl insgesamt,13.18,674000,Sachbeschädigung §§ 303-305a StGB,12.50,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,4.58
4,2019,W,jugendliche_16_bis_unter_18,23775,3172,-3.76,-0.98,*26*00,Ladendiebstahl insgesamt,29.08,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",17.97,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),14.65,674000,Sachbeschädigung §§ 303-305a StGB,4.76,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",1.02


# Now Create same DataFrame for other age groups and then concat them to df_overview_fed_states

In [18]:
%store df_overview_state_16_to_under_18

Stored 'df_overview_state_16_to_under_18' (DataFrame)
